In [6]:
import requests
import pandas as pd
from pyspark.sql import SparkSession
import time

# Initialize Spark Session
spark_session = SparkSession.builder \
    .appName("PySpark S&P 500 Data Loader") \
    .config("spark.jars", "/Users/matthewyip/bootcamp-python/bootcamp-python/postgresql-42.7.7.jar") \
    .getOrCreate()

# Database connection configuration

#Auto-incrementing ID

jdbc_url = "jdbc:postgresql://localhost:5432/bootcamp_2504p"

connection_properties = {
    "user": "postgres",
    "password": "admin1234",
    "driver": "org.postgresql.Driver"
}

# Call API to load S&P 500 symbols
print("📊 Fetching S&P 500 symbols...")
url = "https://raw.githubusercontent.com/datasets/s-and-p-500-companies/main/data/constituents.csv"
symbols_df = pd.read_csv(url)
symbols = symbols_df["Symbol"].tolist()
print(f"✅ Found {len(symbols)} S&P 500 symbols")
print(f"First 10 symbols: {symbols[:10]}")

# Create the main table once (not for each symbol)
print("\n🗄️ Creating stock_ohlcs table...")
try:
    # Create table in PostgreSQL via JDBC
    create_table_query = """
    CREATE TABLE IF NOT EXISTS stock_ohlcs (
        symbol VARCHAR(10),
        date DATE,
        low LONG,
        high LONG,
        open LONG,
        close LONG,
        volume LONG,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (symbol, date)
    )
    """
    
    # Note: You need to execute this SQL directly in your PostgreSQL database
    print("⚠️ Please run this SQL in your PostgreSQL database:")
    print(create_table_query)
    
except Exception as e:
    print(f"❌ Error creating table: {e}")

def fetch_stock_data(symbol, start_period="1657237004", end_period="1751931404"):
    """
    Fetch stock data for a given symbol using proper error handling
    """
    try:
        # Build API URL using f-string for better syntax highlighting
        api_url = f"https://query1.finance.yahoo.com/v8/finance/chart/{symbol}?period1={start_period}&period2={end_period}&interval=1d&events=history"
        
        # Set proper headers
        headers = {
            "User-Agent": "Mozilla/5.0"
        }
        
        print(f"🔄 Fetching data for {symbol}...")
        response = requests.get(api_url, headers=headers)
        response.raise_for_status()  # Raise exception for bad status codes
        
        data = response.json()
        
        # Validate response structure
        if "chart" not in data or not data["chart"]["result"]:
            print(f"⚠️ No data available for {symbol}")
            return None
            
        result = data["chart"]["result"][0]
        
        # Extract price data
        timestamp = pd.Series(result["timestamp"])
        quote = result["indicators"]["quote"][0]
        
        # Create DataFrame
        stock_df = pd.DataFrame({
            "symbol": [symbol] * len(timestamp),
            "date": pd.to_datetime(timestamp, unit="s").dt.date,
            "low": pd.Series(quote["low"]),
            "high": pd.Series(quote["high"]),
            "open": pd.Series(quote["open"]),
            "close": pd.Series(quote["close"]),
            "volume": pd.Series(quote["volume"])
        })
        
        # Handle null values
        stock_df = stock_df.fillna(method='ffill').fillna(method='bfill')
        
        print(f"✅ Successfully fetched {len(stock_df)} records for {symbol}")
        return stock_df
        
    except requests.exceptions.RequestException as e:
        print(f"❌ Network error for {symbol}: {e}")
        return None
    except KeyError as e:
        print(f"❌ Data structure error for {symbol}: {e}")
        return None
    except Exception as e:
        print(f"❌ Unexpected error for {symbol}: {e}")
        return None

def save_to_database(dataframe, symbol):
    """
    Save DataFrame to PostgreSQL database using Spark
    """
    try:
        # Convert to Spark DataFrame
        spark_df = spark_session.createDataFrame(dataframe)
        
        # Write to database
        spark_df.write.jdbc(
            url=jdbc_url,
            table="stock_ohlcs",
            mode="append",
            properties=connection_properties
        )
        
        print(f"💾 Saved {len(dataframe)} records for {symbol} to database")
        return True
        
    except Exception as e:
        print(f"❌ Database error for {symbol}: {e}")
        return False

# Process symbols with proper error handling and rate limiting
print(f"\n🚀 Starting to process {len(symbols)} symbols...")

successful_count = 0
failed_symbols = []

# Process ALL symbols (removed [:5] limit)
for i, symbol in enumerate(symbols, 1):
    print(f"\n--- Processing {symbol} ({i}/{len(symbols)}) ---")
    
    # Fetch stock data
    stock_data = fetch_stock_data(symbol)
    
    if stock_data is not None:
        # Save to database
        if save_to_database(stock_data, symbol):
            successful_count += 1
        else:
            failed_symbols.append(symbol)
    else:
        failed_symbols.append(symbol)
    
    # Rate limiting - pause between requests
    if i < len(symbols):
        print("⏳ Waiting 1 second...")
        time.sleep(1)

# Summary
print(f"\n🎉 Processing completed!")
print(f"✅ Successfully processed: {successful_count} symbols")
print(f"❌ Failed symbols: {len(failed_symbols)}")

if failed_symbols:
    print(f"Failed symbols: {failed_symbols}")

print(f"\n📈 Stock data loaded into 'stock_ohlcs' table!")

📊 Fetching S&P 500 symbols...
✅ Found 502 S&P 500 symbols
First 10 symbols: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

🗄️ Creating stock_ohlcs table...
⚠️ Please run this SQL in your PostgreSQL database:

    CREATE TABLE IF NOT EXISTS stock_ohlcs (
        symbol VARCHAR(10),
        date DATE,
        low LONG,
        high LONG,
        open LONG,
        close LONG,
        volume LONG,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP,
        PRIMARY KEY (symbol, date)
    )
    

🚀 Starting to process 502 symbols...

--- Processing MMM (1/502) ---
🔄 Fetching data for MMM...
✅ Found 502 S&P 500 symbols
First 10 symbols: ['MMM', 'AOS', 'ABT', 'ABBV', 'ACN', 'ADBE', 'AMD', 'AES', 'AFL', 'A']

🗄️ Creating stock_ohlcs table...
⚠️ Please run this SQL in your PostgreSQL database:

    CREATE TABLE IF NOT EXISTS stock_ohlcs (
        symbol VARCHAR(10),
        date DATE,
        low LONG,
        high LONG,
        open LONG,
        close LONG,
      

/var/folders/yd/hjy7_qgx7516bfss7sldck_h0000gn/T/ipykernel_900/1041324049.py:99: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_df = stock_df.fillna(method='ffill').fillna(method='bfill')


✅ Successfully fetched 752 records for MMM
💾 Saved 752 records for MMM to database
⏳ Waiting 1 second...

--- Processing AOS (2/502) ---
🔄 Fetching data for AOS...

--- Processing AOS (2/502) ---
🔄 Fetching data for AOS...


/var/folders/yd/hjy7_qgx7516bfss7sldck_h0000gn/T/ipykernel_900/1041324049.py:99: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_df = stock_df.fillna(method='ffill').fillna(method='bfill')


✅ Successfully fetched 752 records for AOS
💾 Saved 752 records for AOS to database
⏳ Waiting 1 second...

--- Processing ABT (3/502) ---
🔄 Fetching data for ABT...

--- Processing ABT (3/502) ---
🔄 Fetching data for ABT...


...

/var/folders/yd/hjy7_qgx7516bfss7sldck_h0000gn/T/ipykernel_900/1041324049.py:99: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_df = stock_df.fillna(method='ffill').fillna(method='bfill')


✅ Successfully fetched 752 records for BMY
💾 Saved 752 records for BMY to database
⏳ Waiting 1 second...

--- Processing AVGO (72/502) ---
🔄 Fetching data for AVGO...
✅ Successfully fetched 752 records for AVGO

--- Processing AVGO (72/502) ---
🔄 Fetching data for AVGO...
✅ Successfully fetched 752 records for AVGO


/var/folders/yd/hjy7_qgx7516bfss7sldck_h0000gn/T/ipykernel_900/1041324049.py:99: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  stock_df = stock_df.fillna(method='ffill').fillna(method='bfill')


💾 Saved 752 records for AVGO to database
⏳ Waiting 1 second...


KeyboardInterrupt: 